In [1]:
import numpy as np
import pandas as pd

## To generate a Source Extractor catalog:

1. Follow instructions here to install Source Extractor (https://sextractor.readthedocs.io/en/latest/Installing.html)

2. Copy `default.conv`, `default.nnw`, `default.param`, `default.sex` into your working directory with the image to extract (or generate these files yourself)

3. Edit `default.sex` to change the CATALOG_NAME parameter to something meaningful (e.g., FRB20190608B_rband.cat)

4. Run command `sex $filename -c default.sex` to extract sources

5. Proceed with notebook to query region of interest and save a ds9 region file

In [ ]:
def query_cat(obj_name, RA_host_deg, dec_host_deg, radius_arcsec):

    """
    Function to load a source extractor catalog and save ds9 region files in a box about a given position

    Inputs:
        obj_name: name of SE catalog (prior to .cat)
        RA_host_deg: RA of position to query about in degrees
        dec_host_deg: dec of position to query about in degrees
        radius_arcsec: search radius to query in arcsec

    Output:
        ds9 region file and small catalog about position
    """

    # load in SE catalog
    df = pd.read_csv('{}.cat'.format(obj_name), skiprows=25, sep='\s+', names=['NUMBER', 'MAGERR_ISO', 'MAG_APER', 'MAGERR_APER', 'MAG_AUTO', 'MAGERR_AUTO', 'MAG_BEST', 'MAGERR_BEST', 'ISOAREA_IMAGE', 'X_IMAGE', 'Y_IMAGE', 'ALPHA_J2000', 'DELTA_J2000', 'A_IMAGE', 'B_IMAGE', 'ERRA_IMAGE', 'ERRB_IMAGE', 'ERRTHETA_IMAGE', 'THETA_IMAGE', 'A_WORLD', 'B_WORLD', 'THETA_J2000', 'FWHM_IMAGE', 'FWHM_WORLD', 'CLASS_STAR'])

    # isolate columns of interest
    RA_vals = df['ALPHA_J2000']
    dec_vals = df['DELTA_J2000']
    X_vals = df['X_IMAGE']
    Y_vals = df['Y_IMAGE']
    A = df['A_IMAGE']
    B = df['B_IMAGE']
    theta = df['THETA_IMAGE']
    errA = df['ERRA_IMAGE']
    errB = df['ERRB_IMAGE']
    errtheta = df['ERRTHETA_IMAGE'] # deg E of N
    star = df['CLASS_STAR']
    mag = df['MAG_AUTO']
    mag_err = df['MAGERR_AUTO']

    # search SE catalog about host position
    RA_cut = []
    dec_cut = []
    X_cut = []
    Y_cut = []
    A_cut = []
    B_cut = []
    theta_cut = []
    errA_cut = []
    errB_cut = []
    errtheta_cut = []
    star_cut = []
    mag_cut = []
    magerr_cut = []

    radius_deg = radius_arcsec/3600

    for (RA,DEC,X,Y,eA,eB,eth,st,m,me,a,b,t) in zip(RA_vals,dec_vals,X_vals,Y_vals,errA,errB,errtheta,star,mag,mag_err,A,B,theta):
        if ((RA < RA_host_deg+radius_deg) and (RA > RA_host_deg-radius_deg)) and ((DEC < dec_host_deg+radius_deg) and (DEC > dec_host_deg-radius_deg)):
            RA_cut.append(RA)
            dec_cut.append(DEC)
            X_cut.append(X)
            Y_cut.append(Y)
            A_cut.append(a)
            B_cut.append(b)
            theta_cut.append(t)
            errA_cut.append(eA)
            errB_cut.append(eB)
            errtheta_cut.append(eth)
            star_cut.append(st)
            mag_cut.append(m)
            magerr_cut.append(me)

    # save to a ds9 region file
    np.savetxt('{0}-catalog-{1}arcsec.reg'.format(obj_name,radius_arcsec), np.column_stack((RA_cut, dec_cut)), fmt='circle(%1.14f,%1.14f,0.5") # color=green')

    # add header info for ds9
    copy = open('{0}-catalog-{1}arcsec.reg'.format(obj_name,radius_arcsec), 'r')
    text = copy.read() # save current text
    copy = open('{0}-catalog-{1}arcsec.reg'.format(obj_name,radius_arcsec), 'w')
    copy.write('#Region file format: DS9 version 4.1\n')
    copy.write('fk5\n')
    copy.write(text) # put text after header
    copy.close()

    # save queried region to a catalog file
    np.savetxt('{0}-catalog-{1}arcsec.cat'.format(obj_name,radius_arcsec), np.column_stack((RA_cut, dec_cut, A_cut, B_cut, theta_cut, errA_cut, errB_cut, errtheta_cut, star_cut, mag_cut, magerr_cut)))

    # add header info for small catalog
    copy = open('{0}-catalog-{1}arcsec.cat'.format(obj_name,radius_arcsec), 'r')
    text = copy.read() # save current text
    copy = open('{0}-catalog-{1}arcsec.cat'.format(obj_name,radius_arcsec), 'w')
    copy.write('#  1 ALPHA_J2000            Right ascension of barycenter (J2000)                      [deg]\n')
    copy.write('#  2 DELTA_J2000            Declination of barycenter (J2000)                          [deg]\n')
    copy.write('#  3 A_IMAGE                Profile RMS along major axis                               [pixel]\n')
    copy.write('#  4 B_IMAGE                Profile RMS along minor axis                               [pixel]\n')
    copy.write('#  5 THETA_IMAGE            Position angle (CCW/x)                                     [deg]\n')
    copy.write('#  6 ERRA_IMAGE             RMS position error along major axis                        [pixel]\n')
    copy.write('#  7 ERRB_IMAGE             RMS position error along minor axis                        [pixel]\n')
    copy.write('#  8 ERRTHETA_IMAGE         Error ellipse position angle (CCW/x)                       [deg]\n')
    copy.write('#  9 CLASS_STAR             S/G classifier output \n')
    copy.write('#  10 MAG_AUTO               Kron-like elliptical aperture magnitude                    [mag]\n')
    copy.write('#  11 MAGERR_AUTO            RMS error for AUTO magnitude                               [mag]\n')
    copy.write(text)
    copy.close()
                                    
    print('Catalog {0} queried about ({1}, {2}) with {3} arcsec radius box'.format(obj_name, RA_host_deg, dec_host_deg, radius_arcsec))

    return

In [5]:
# query catalog and save files
query_cat('example_files/FRB20190608B_rband', 334.020417, -7.89883, 30)

Catalog example_files/FRB20190608B_rband queried about (334.020417, -7.89883) with 30 arcsec radius box


## Description of catalog columns

1. ALPHA_J2000: Right ascension in degrees

2. DELTA_J2000: Declination in degrees

3. A_IMAGE: Semi-major axis of isophotal ellipse in pixels

4. B_IMAGE: Semi-minor axis of isophotal ellipse in pixels

5. THETA_IMAGE: Position angle of isophotal ellipse counterclockwise from image x-axis

6. ERRA_IMAGE: Semi-major axis of positional uncertainty ellipse in pixels

7. ERRB_IMAGE: Semi-minor axis of positional unceratinty ellipse in pixels

8. ERRTHETA_IMAGE: Position angle of unceratinty ellipse counterclockwise from image x-axis

9. CLASS_STAR: Star/galaxy classifier. Values close to zero are likely galaxy; values close to one are likely star

10. MAG_AUTO: Kron magnitude aperture photometry

11. MAGERR_AUTO: Uncertainty on Kron magnitude 

## Calculating positional uncertainty of host coordinates

Uses `ERRA_IMAGE`, `ERRB_IMAGE`, `ERRTHETA_IMAGE` from catalog and pixelscale from image header

In [6]:
def unc_to_RADec(errA, errB, errtheta, pix_scale):
    """
    Converts position uncertainty ellipse into RA/Dec components in arcsec
    Inputs:
        errA [float]: semimajor axis of uncertainty ellipse (pixels)
        errB [float]: semiminor axis of uncertainty ellipse (pixels)
        errtheta [float]: position angle of uncertainty ellipse (deg E of N)
        pix_scale [float]: pixel scale of image (arcsec/pixels)
    Outputs:
        dRA [float]: RA uncertainty of host position (arcsec)
        dDec [float]: Dec uncertainty of host position (arcsec)
        dtotal [float]: Combined uncertainty on host position in quadrature (milliarcsec)
    """
    dRA_pix = np.sqrt(errA**2*np.cos(np.deg2rad(errtheta))**2 + errB**2*np.sin(np.deg2rad(errtheta))**2)
    dDec_pix = np.sqrt(errA**2*np.sin(np.deg2rad(errtheta))**2 + errB**2*np.cos(np.deg2rad(errtheta))**2)
    dRA = dRA_pix * pix_scale
    dDec = dDec_pix * pix_scale
    dtotal = np.sqrt(dRA**2 + dDec**2)
    return dRA, dDec, dtotal

In [ ]:
# edit cell with values from catalog corresponding to your object of interest

errA_host = 2.219000000000000139e-02        # column 6 of catalog
errB_host = 1.948000000000000079e-02        # column 7 of catalog
errtheta_host = 5.585000000000000142e+01    # column 8 of catalog
pixscale = 0.262                            # arcsec/pix, see image header

ra, dec, tot = unc_to_RADec(errA_host, errB_host, errtheta_host, pixscale)
print('Uncertainty in host position: RA={0}", Dec={1}", total={2}"'.format(ra,dec,tot))

Uncertainty in host position: RA=0.005337707772662839", Dec=0.005599757116130549", total=0.007736175025553649"
